<a href="https://colab.research.google.com/github/ParthPatel101/iSPY/blob/main/iSPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This program uses LSTM (Long Short Term Memory) to predict the stock price of the SPY 500 index using past data.

In [3]:
import math
import pandas_datareader as web
import datetime as dt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [8]:
start = dt.datetime(1993, 1, 1)
end = dt.datetime.today()
df = web.DataReader("SPY", data_source = "yahoo", start = start, end = end)
print(df)

RemoteDataError: ignored

In [ ]:
plt.figure(figsize=(18,8))
plt.title("SPY Price History")
plt.plot(df["Close"])
plt.xlabel("Year", fontsize = 18)
plt.ylabel("Price (USD)", fontsize = 18)
plt.show()

In [ ]:
data = df.filter(["Close"])
dataset = data.values
training_data_length = math.ceil(len(dataset) * 0.95)
print(training_data_length)

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data = scaled_data[0:training_data_length]
x_train = []
y_train = []
for i in range(60, training_data_length):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i <= 60:
    print(x_train)
    print(y_train)
    print()

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1],1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer = "adam", loss = "mean_squared_error")

In [ ]:
model.fit(x_train, y_train, batch_size = 1, epochs = 1)

In [ ]:
test_data = scaled_data[training_data_length - 60:]
x_test = []
y_test = dataset[training_data_length:]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
train = data[:training_data_length]
valid = data[training_data_length:]
valid["Predictions"] = predictions
plt.figure(figsize = (16,8))
plt.title("SPY Price History")
plt.xlabel("Year", fontsize = 18)
plt.ylabel("Price (USD)", fontsize = 18)
plt.plot(train["Close"])
plt.plot(valid[["Close", "Predictions"]])
plt.legend(["Train", "Val", "Predictions"], loc = "best")
plt.show()

In [ ]:
valid

In [ ]:
new_df = df.filter(["Close"])
last_60_days = new_df[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)
new_x_test = []
new_x_test.append(last_60_days_scaled)
new_x_test = np.array(new_x_test)
new_x_test = np.reshape(new_x_test, (new_x_test.shape[0], new_x_test.shape[1], 1))
pred_price = model.predict(new_x_test)
pred_price = scaler.inverse_transform(pred_price)
print("Predicted close price for SPY for {}: ${:0.2f}".format((df.index[-1] +  dt.timedelta(days = 1)).strftime('%Y-%m-%d'), round(pred_price[0][0], 2)))

Predicted close price for SPY for 2022-07-16: $390.36
